# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>

#### <center> **Final Project: Machine Learning** </center>
---

**Date**: October, 2025

**Student Name**: Juan Alonso

**Professor**: Pablo Camarillo Ramirez

## Machine Learning algorithm to use

En este proyecto se quiere hacer clustering de clientes de tarjeta de crédito.  
La idea es agrupar a los clientes según su forma de usar la tarjeta: cuánto gastan, cuánto deben, cuánto adelantan en efectivo, cuánto pagan del mínimo, etc.

Se eligió K-Means porque:

- Es un algoritmo de aprendizaje no supervisado.
- Permite crear segmentos de clientes que tienen comportamientos parecidos.
- Es más fácil de entender, cada cluster tiene un centroide que resume el promedio del grupo.
- Spark MLlib ya trae una implementación de K-Means que vimos en clase.

Con este modelo se busca encontrar grupos como:
- Clientes que gastan mucho y pagan a tiempo.
- Clientes que usan mucho adelantos en efectivo.
- Clientes con poco uso de la tarjeta.

## Dataset Description

Para este proyecto se usa el dataset "Credit Card Dataset for Clustering" de Kaggle  
Link: https://www.kaggle.com/datasets/arjunbhasin2013/ccdata.

Fuente: Kaggle – Credit Card Dataset for Clustering  
Tamaño:
- 8,950 clientes.
- 18 columnas con información de uso de tarj eta.

Algunas columnas importantes son:

- `BALANCE`: saldo promedio de la tarjeta.
- `PURCHASES`: total de compras hechas.
- `ONEOFF_PURCHASES`: compras grandes de una sola exhibición.
- `INSTALLMENTS_PURCHASES`: compras en pagos.
- `CASH_ADVANCE`: adelantos de efectivo.
- `CREDIT_LIMIT`: límite de crédito.
- `PAYMENTS`: pagos realizados.
- `MINIMUM_PAYMENTS`: pagos mínimos hechos.

Como es un problema de clustering, lo que importa es cuántas dimensiones vamos a usar para agrupar. En este caso se seleccionan las 8 columnas identificadas como variables numéricas para usarse como features principales; así que el problema de clustering se hace en un espacio de 8 dimensiones.


In [6]:
# Schema definition
schema_columns = [
    ("CUST_ID", "string"),
    ("BALANCE", "double"),
    ("BALANCE_FREQUENCY", "double"),
    ("PURCHASES", "double"),
    ("ONEOFF_PURCHASES", "double"),
    ("INSTALLMENTS_PURCHASES", "double"),
    ("CASH_ADVANCE", "double"),
    ("PURCHASES_FREQUENCY", "double"),
    ("ONEOFF_PURCHASES_FREQUENCY", "double"),
    ("PURCHASES_INSTALLMENTS_FREQUENCY", "double"),
    ("CASH_ADVANCE_FREQUENCY", "double"),
    ("CASH_ADVANCE_TRX", "double"),
    ("PURCHASES_TRX", "double"),
    ("CREDIT_LIMIT", "double"),
    ("PAYMENTS", "double"),
    ("MINIMUM_PAYMENTS", "double"),
    ("PRC_FULL_PAYMENT", "double"),
    ("TENURE", "double")
]

credit_schema = SparkUtils.generate_schema(schema_columns)

DATA_PATH = "/opt/spark/work-dir/data/ml/juanalonso/credit_card.csv"

df_credit = spark.read \
    .option("header", "true") \
    .schema(credit_schema) \
    .csv(DATA_PATH)

print("--- Esquema del Dataset ---")
df_credit.printSchema()

print("\n--- Tamaño del Dataset ---")
print(f"Total de filas: {df_credit.count()}")
print(f"Total de columnas: {len(df_credit.columns)}")

--- Esquema del Dataset ---
root
 |-- CUST_ID: string (nullable = true)
 |-- BALANCE: double (nullable = true)
 |-- BALANCE_FREQUENCY: double (nullable = true)
 |-- PURCHASES: double (nullable = true)
 |-- ONEOFF_PURCHASES: double (nullable = true)
 |-- INSTALLMENTS_PURCHASES: double (nullable = true)
 |-- CASH_ADVANCE: double (nullable = true)
 |-- PURCHASES_FREQUENCY: double (nullable = true)
 |-- ONEOFF_PURCHASES_FREQUENCY: double (nullable = true)
 |-- PURCHASES_INSTALLMENTS_FREQUENCY: double (nullable = true)
 |-- CASH_ADVANCE_FREQUENCY: double (nullable = true)
 |-- CASH_ADVANCE_TRX: double (nullable = true)
 |-- PURCHASES_TRX: double (nullable = true)
 |-- CREDIT_LIMIT: double (nullable = true)
 |-- PAYMENTS: double (nullable = true)
 |-- MINIMUM_PAYMENTS: double (nullable = true)
 |-- PRC_FULL_PAYMENT: double (nullable = true)
 |-- TENURE: double (nullable = true)


--- Tamaño del Dataset ---
Total de filas: 8950
Total de columnas: 18


## ML Training process

Antes de entrenar el modelo, se hacen algunos pasos básicos:

1. Se eliminan filas que tengan valores nulos en columnas importantes, para no afectar el entrenamiento.
2. Se seleccionan solo las columnas numéricas que se van a usar como features.
3. Se usa VectorAssembler para juntar las 8 columnas numéricas en una sola columna llamada features.

Se eligió K-Means porque:
- Es sencillo de entender.
- Funciona bien cuando se quiere agrupar puntos en base a distancia.
- Es parte de Spark MLlib, así que se integra fácil al pipeline.

Hiperparámetros usados:

- `k`: número de clusters.  
  En lugar de poner un valor fijo, se probó un rango de k (de 2 a 10).  
  Para cada k se entrena un modelo y se calcula el Silhouette Score.  
  Se elige el k que tenga el Silhouette más alto.  
  Se eligió esta forma porque así el modelo decide cuántos grupos tienen más sentido.

- `seed`: valor de semilla aleatoria.  
  Se usa `setSeed(42)` para que el resultado sea reproducible.  
  Esto quiere decir que, si se vuelve a correr el notebook, el modelo da los mismos clusters.

- `featuresCol`: nombre de la columna de entrada para el modelo.  
  Se usa `"features"`, que es la salida del `VectorAssembler`.

Con esto se arma el pipeline simple:
datos limpios -> vector assembler -> múltiples modelos K-Means -> elegir mejor k -> entrenar modelo final.

In [4]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from juanalonso.spark_utils import SparkUtils

from pyspark.ml.clustering import KMeans, KMeansModel
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.feature import VectorAssembler

spark = SparkSession.builder \
    .appName("ML_FinalProject_JuanAlonso") \
    .master("local[*]") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("WARN")

print("SparkSession iniciada.")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/25 03:31:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


SparkSession iniciada.


In [8]:
from pyspark.sql import functions as F

# Data cleanup
feature_cols = [
    "BALANCE",
    "PURCHASES",
    "ONEOFF_PURCHASES",
    "INSTALLMENTS_PURCHASES",
    "CASH_ADVANCE",
    "CREDIT_LIMIT",
    "PAYMENTS",
    "MINIMUM_PAYMENTS"
]

df_clean = df_credit.dropna(subset=feature_cols)

print(f"Filas despues de dropna: {df_clean.count()}")

vector_assembler = VectorAssembler(
    inputCols=feature_cols,
    outputCol="features"
)

assembled_df = vector_assembler.transform(df_clean)

print("\n--- Ejemplo de la columna features ---")
assembled_df.select("features").show(5, truncate=False)

Filas despues de dropna: 8636

--- Ejemplo de la columna features ---
+--------------------------------------------------------------------+
|features                                                            |
+--------------------------------------------------------------------+
|[40.900749,95.4,0.0,95.4,0.0,1000.0,201.802084,139.509787]          |
|[3202.467416,0.0,0.0,0.0,6442.945483,7000.0,4103.032597,1072.340217]|
|[2495.148862,773.17,773.17,0.0,0.0,7500.0,622.066742,627.284787]    |
|[817.714335,16.0,16.0,0.0,0.0,1200.0,678.334763,244.791237]         |
|[1809.828751,1333.28,0.0,1333.28,0.0,1800.0,1400.05777,2407.246035] |
+--------------------------------------------------------------------+
only showing top 5 rows


In [10]:
# Get best Silhouette Score

k_values = range(2, 11)
silhouette_scores = {}

evaluator = ClusteringEvaluator(
    featuresCol="features",
    predictionCol="prediction",
    metricName="silhouette"
)

print("\n--- Buscando k optimo ---")
for k in k_values:
    kmeans = KMeans().setK(k).setSeed(42).setFeaturesCol("features")
    model = kmeans.fit(assembled_df)
    
    predictions = model.transform(assembled_df)
    silhouette = evaluator.evaluate(predictions)
    silhouette_scores[k] = silhouette
    
    print(f"k = {k}, Silhouette Score = {silhouette:.4f}")

optimal_k = max(silhouette_scores, key=silhouette_scores.get)
best_score = silhouette_scores[optimal_k]

print("\n--- Resultados ---")
print(f"El mejor k es: {optimal_k} con Silhouette = {best_score:.4f}")



--- Buscando k optimo ---
k = 2, Silhouette Score = 0.6441
k = 3, Silhouette Score = 0.5983
k = 4, Silhouette Score = 0.6074
k = 5, Silhouette Score = 0.5198
k = 6, Silhouette Score = 0.5285
k = 7, Silhouette Score = 0.5341
k = 8, Silhouette Score = 0.4446
k = 9, Silhouette Score = 0.5410
k = 10, Silhouette Score = 0.4970

--- Resultados ---
El mejor k es: 2 con Silhouette = 0.6441


In [ ]:
# Save the model

MODEL_PATH = "/opt/spark/work-dir/data/mlmodels/final_project/kmeans_credit_card_clusters"

print(f"\nEntrenando modelo final con k={optimal_k} ...")
kmeans_final = KMeans().setK(optimal_k).setSeed(42).setFeaturesCol("features")
final_model = kmeans_final.fit(assembled_df)

final_model.write().overwrite().save(MODEL_PATH)
print(f"Modelo final guardado en: {MODEL_PATH}")



Entrenando modelo final con k=2 ...


Modelo final guardado en: /opt/spark/work-dir/data/mlmodels/final_project/kmeans_credit_card_clusters


## ML Evaluation

Como este es un problema de clustering, no hay etiquetas reales.  
Por eso no se puede usar Accuracy, Precision o Recall.

Para evaluar el modelo se usa el Silhouette Score, que mide dos cosas:

- Qué tan cerca está cada punto de los puntos de su propio cluster, la cohesión.
- Qué tan lejos está de los puntos de otros clusters, la separación.

El valor va de -1 a 1:

- Cerca de 1: clusters bien formados y separados.
- Cerca de 0: clusters que se empalman.
- cerca de -1: mala asignación de clusters.

En este proyecto se usa el Silhouette de dos formas:

1. Para elegir el mejor k en la fase de entrenamiento.
2. Para evaluar el modelo final cargado desde disco.

También se muestran:

- Un ejemplo de clientes con su cluster asignado.
- El conteo de clientes por cluster.
- Los centroides de cada cluster, que ayudan a interpretar cada segmento.

In [ ]:
# Load the model
loaded_model = KMeansModel.load(MODEL_PATH)
print(f"Modelo K-Means cargado. k = {loaded_model.getK()}")

# Get predictions
predictions = loaded_model.transform(assembled_df)

print("\n--- Muestra de predicciones ---")
predictions.select(
    "CUST_ID",
    "BALANCE",
    "PURCHASES",
    "CASH_ADVANCE",
    "CREDIT_LIMIT",
    "PAYMENTS",
    "MINIMUM_PAYMENTS",
    "prediction"
).show(10, truncate=False)

# Evaluate Silhouette Score
final_silhouette = evaluator.evaluate(predictions)

print("\n--- Evaluacion final ---")
print(f"Silhouette Score: {final_silhouette:.4f}")


Modelo K-Means cargado. k = 2

--- Muestra de predicciones ---
+-------+-----------+---------+------------+------------+-----------+----------------+----------+
|CUST_ID|BALANCE    |PURCHASES|CASH_ADVANCE|CREDIT_LIMIT|PAYMENTS   |MINIMUM_PAYMENTS|prediction|
+-------+-----------+---------+------------+------------+-----------+----------------+----------+
|C10001 |40.900749  |95.4     |0.0         |1000.0      |201.802084 |139.509787      |0         |
|C10002 |3202.467416|0.0      |6442.945483 |7000.0      |4103.032597|1072.340217     |1         |
|C10003 |2495.148862|773.17   |0.0         |7500.0      |622.066742 |627.284787      |0         |
|C10005 |817.714335 |16.0     |0.0         |1200.0      |678.334763 |244.791237      |0         |
|C10006 |1809.828751|1333.28  |0.0         |1800.0      |1400.05777 |2407.246035     |0         |
|C10007 |627.260806 |7091.01  |0.0         |13500.0     |6354.314328|198.065894      |1         |
|C10008 |1823.652743|436.2    |0.0         |2300.0     

In [13]:
print("\n--- Conteo de clientes por cluster ---")
predictions.groupBy("prediction").count().orderBy("prediction").show()

print("\n--- Centroides de cada cluster ---")
centers = loaded_model.clusterCenters()
for idx, center in enumerate(centers):
    print(f"Clúster {idx}:")
    print(f"  BALANCE promedio:                {center[0]:.2f}")
    print(f"  PURCHASES promedio:              {center[1]:.2f}")
    print(f"  ONEOFF_PURCHASES promedio:       {center[2]:.2f}")
    print(f"  INSTALLMENTS_PURCHASES promedio: {center[3]:.2f}")
    print(f"  CASH_ADVANCE promedio:           {center[4]:.2f}")
    print(f"  CREDIT_LIMIT promedio:           {center[5]:.2f}")
    print(f"  PAYMENTS promedio:               {center[6]:.2f}")
    print(f"  MINIMUM_PAYMENTS promedio:       {center[7]:.2f}\n")



--- Conteo de clientes por cluster ---
+----------+-----+
|prediction|count|
+----------+-----+
|         0| 6797|
|         1| 1839|
+----------+-----+


--- Centroides de cada cluster ---
Clúster 0:
  BALANCE promedio:                978.04
  PURCHASES promedio:              646.17
  ONEOFF_PURCHASES promedio:       338.44
  INSTALLMENTS_PURCHASES promedio: 308.01
  CASH_ADVANCE promedio:           550.28
  CREDIT_LIMIT promedio:           3068.29
  PAYMENTS promedio:               1075.95
  MINIMUM_PAYMENTS promedio:       610.34

Clúster 1:
  BALANCE promedio:                3904.52
  PURCHASES promedio:              2427.20
  ONEOFF_PURCHASES promedio:       1589.74
  INSTALLMENTS_PURCHASES promedio: 837.87
  CASH_ADVANCE promedio:           2634.84
  CREDIT_LIMIT promedio:           9895.38
  PAYMENTS promedio:               4403.23
  MINIMUM_PAYMENTS promedio:       1802.96



In [14]:
sc.stop()
spark.stop()